##### Due to the possibility of information of data leaks, all outputs are cleared. 

Feature names are not cleared because they are commonly used AIS data feature name.

In [ ]:
import pandas as pd
import numpy as np
import os

Dataset feature explanation:

mmsi: unique code for each ship

positionsourcedate: time of the measurement

latitude: latitude

longitude: longitude

navstatuscode: voyage status

To build trajectory data from the dataset, mmsi, positionsourcedate, latitude, longitude are used.

##### Load dataset (non-public data is used, so I cannot give data)

In [ ]:
df = pd.DataFrame()
for i in os.listdir("AIS"):
    temp = pd.read_excel(f"AIS/{i}")
    df = pd.concat([df, temp])

In [ ]:
df = df[["mmsi", "longitude", "latitude", "positionsourcedate", 'navstatuscode']]
df = df.sort_values(['mmsi', 'positionsourcedate'])

In [ ]:
df['positionsourcedate'] = pd.to_datetime(df['positionsourcedate'])

# only use year 2023
df = df[df['positionsourcedate'].dt.year == 2023]

In [ ]:
# only use Jan
df[df['positionsourcedate'].dt.month.isin([1])]
# change positionsourcedate to hour
df["hours"] = ((df['positionsourcedate'].dt.days_in_month + df["positionsourcedate"].dt.day) * 24 + df["positionsourcedate"].dt.hour)

In [ ]:
# when there is more than one measurement in an hour, use first one
df = df.drop_duplicates(subset=['mmsi', 'hours'])

devide data into each observation

observation: a voyage of one ship

use navi code, devide the data (use code that means ship is not moving)

In [ ]:
df = df.sort_values(by=["mmsi", "positionsourcedate"], inplace=True)

In [ ]:
df.columns

In [ ]:
# distinguish voyage of ship
ids = df['mmsi'].unique()
res = []
count = 0
for i in range(len(df.index)-1):
    # check mmsi is same
    if df.iloc[i, 0] == df.iloc[i+1, 0]:
        # check navstatuscode to distinguish voyages by using at port status
        if df.iloc[i, 4] in [1, 5, 15]:
            res.append(False)
            count += 1
        else:
            res.append(f"{df.iloc[i, 0]}_{count}")
    else:
        count = 0
        res.append(f"{df.iloc[i, 0]}_{count}")
res.append(f"{df.iloc[-1, 0]}_{count}")

df['voyage'] = res
df = df[df['voyage'] != False]

In [ ]:
vid, num = np.unique(df['voyage'], return_counts=True)

In [ ]:
# use data sample with more than 50 time points
T = 50
vid = vid[num >= T]
df = df[df['voyage'].isin(vid)]

In [ ]:
len(vid)

In [ ]:
df = df[['voyage', 'longitude', 'latitude', 'hours']]

In [ ]:
# refine starting point to 0
df = df.sort_values(['voyage', 'hours'])
whole = pd.DataFrame()
for i in vid:
    temp = df[df['voyage'] == i]
    base = temp.iloc[0, 3]
    temp.loc[:, 'hours'] = temp.loc[:, 'hours'] - base
    whole = pd.concat([whole, temp])

In [ ]:
df1 = whole
df1

In [ ]:
# remove sample contains missing time points
cont = []
ll = []
for i in vid:
    count = 0
    num = 0
    temp = df1[df1['voyage'] == i]
    for h in range(len(temp)):
        if temp.iloc[h, 3] - count > 1:
            num = 0
        else:
            count = temp.iloc[h, 3]
            num += 1
            if num == T:
                ll.append(h-T+1)
                cont.append(i)

In [ ]:
len(cont)

In [ ]:
df = df[df['voyage'].isin(cont)]

In [ ]:
whole = pd.DataFrame()
for k, i in enumerate(cont):
    temp = df[df['voyage'].isin([i])]
    temp = temp.iloc[ll[k]:ll[k]+T, :]
    whole = pd.concat([whole, temp])

In [ ]:
whole

In [ ]:
# refine starting point to 0
whole_ = pd.DataFrame()
for i in cont:
    temp = whole[whole['voyage'] == i]
    base = temp.iloc[0, 3]
    temp.loc[:, 'hours'] = temp.loc[:, 'hours'] - base
    whole_ = pd.concat([whole_, temp])

In [ ]:
# save dataset
import pickle
with open("AIS/data.pickle", 'wb') as f:
    pickle.dump(whole_, f)
